# Example IMC analysis with Morpheus

## Step 0 (optional): set seed for reproducibility 

In [1]:
from lightning.pytorch import seed_everything
seed_everything(42)

%reload_ext autoreload
%autoreload 2

/central/home/zwang2/.cache/pypoetry/virtualenvs/morpheus-spatial-ndDQRg-x-py3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 42


## Step 1: Creating a SpatialDataset Object

In this tutorial, we will start by creating a `SpatialDataset` object, which will hold all relevant information about the dataset we will be working with. 

### Prerequisites

To create a `SpatialDataset` object, you will need:
- The path to the CSV file containing all single-cell expression information
- A list of channel names

### CSV File Structure

The expected structure of the CSV file is as follows:
- Each row corresponds to a single cell
- Columns for each channel name, with expression values specified
- Five additional columns with the following names and information:

| Column Name         | Description                               | Datatype    |
|---------------------|-------------------------------------------|-------------|
| `ImageNumber`       | Unique ID for each image                  | Integer     |
| `PatientID`         | Unique ID for each patient                | Str/Integer |
| `CellType`          | Cell type                                 | Str         |
| `Location_Center_X` | X coordinate of the cell center in micron | Float       |
| `Location_Center_Y` | Y coordinate of the cell center in micron | Float       |

Note: Additional metadata columns beyond these will not be used for the analysis performed in this tutorial.

To create a `SpatialDataset` object, use the following code, remember to replace `'path/to/your/csv/file.csv'` with the actual path to your CSV file.

In [2]:
import morpheus as mp

data_path = "/groups/mthomson/zwang2/IMC/output/danenbergBreast_sz48_pxl3_nc39/replicate/singlecell.csv"  # change to your own directory
dataset = mp.SpatialDataset(input_path=data_path)

2024-05-04 18:52:30,250	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


39 channels inferred from input CSV: ['Histone H3', 'SMA', 'CK5', 'CD38', 'HLA-DR', 'CK8-18', 'CD15', 'FSP1', 'CD163', 'ICOS', 'OX40', 'CD68', 'HER2 (3B5)', 'CD3', 'Podoplanin', 'CD11c', 'PD-1', 'GITR', 'CD16', 'HER2 (D8F12)', 'CD45RA', 'B2M', 'CD45RO', 'FOXP3', 'CD20', 'ER', 'CD8', 'CD57', 'Ki-67', 'PDGFRB', 'Caveolin-1', 'CD4', 'CD31-vWF', 'CXCL12', 'HLA-ABC', 'panCK', 'c-Caspase3', 'DNA1', 'DNA2']
Input path: /groups/mthomson/zwang2/IMC/output/danenbergBreast_sz48_pxl3_nc39/replicate/singlecell.csv
Patch path: /groups/mthomson/zwang2/IMC/output/danenbergBreast_sz48_pxl3_nc39/replicate/patch.h5
Split directory: None
Model path: None
Counterfactual directory: None


## Step 2: patch images and mask cells

In [3]:
patch_size = 16  # Patch size in pixels
pixel_size = 3  # Pixel size in microns
cell_types = ["Tcytotoxic", "Tumor"]  # Specify the cell types of interest
mask_cell_types = ["Tcytotoxic"]
dataset.generate_masked_patch(
    cell_to_mask=mask_cell_types,
    cell_types=cell_types,
    patch_size=patch_size,
    pixel_size=pixel_size,
    save=True,
)

File /groups/mthomson/zwang2/IMC/output/danenbergBreast_sz48_pxl3_nc39/replicate/patch.h5 already exists, existing file loaded


## Step 3: generate data splits for model training

Next, we will need to generate train, validation, and test data splits for model training. We want to stratify our splits by the label we want to predict.

In [ ]:
colname = "Contains_Tcytotoxic"
dataset.generate_data_splits(stratify_by=colname, given_split=[['MB-0282', 'MB-0598', 'MB-0606', 'MB-0190', 'MB-0528', 'MB-0211', 'MB-0442', 'MB-0224', 'MB-0321', 'MB-0248', 'MB-0511', 'MB-0650', 'MB-0369', 'MB-0555', 'MB-0569', 'MB-0570', 'MB-0344', 'MB-0334', 'MB-0556', 'MB-0557', 'MB-0236', 'MB-0317', 'MB-0345', 'MB-0351', 'MB-0353', 'MB-0573', 'MB-0357', 'MB-0869', 'MB-0872', 'MB-0874', 'MB-0877', 'MB-0350', 'MB-0358', 'MB-0882', 'MB-0364', 'MB-0893', 'MB-0895', 'MB-0901', 'MB-0906', 'MB-0884', 'MB-0359', 'MB-0360', 'MB-0361', 'MB-0891', 'MB-0271', 'MB-0272', 'MB-0273', 'MB-0278', 'MB-0268', 'MB-0270', 'MB-0535', 'MB-0294', 'MB-0045', 'MB-0081', 'MB-0114', 'MB-0115', 'MB-0232', 'MB-0897', 'MB-0537', 'MB-0193', 'MB-0482', 'MB-0496', 'MB-0220', 'MB-0221', 'MB-0223', 'MB-0280', 'MB-0214', 'MB-0374', 'MB-0445', 'MB-0229', 'MB-0305', 'MB-0320', 'MB-0542', 'MB-0301', 'MB-0302', 'MB-0304', 'MB-0543', 'MB-0324', 'MB-0547', 'MB-0311', 'MB-0549', 'MB-0240', 'MB-0325', 'MB-0545', 'MB-0532', 'MB-0256', 'MB-0913', 'MB-0258', 'MB-0252', 'MB-0530', 'MB-0269', 'MB-0266', 'MB-0551', 'MB-0552', 'MB-0553', 'MB-0260', 'MB-0261', 'MB-0262', 'MB-0263', 'MB-0056', 'MB-0059', 'MB-0046', 'MB-0068', 'MB-0668', 'MB-0103', 'MB-0106', 'MB-0008', 'MB-0073', 'MB-0079', 'MB-0083', 'MB-0089', 'MB-0097', 'MB-0218', 'MB-0589', 'MB-0591', 'MB-0593', 'MB-0594', 'MB-0595', 'MB-0366', 'MB-0584', 'MB-0587', 'MB-0588', 'MB-0597', 'MB-0599', 'MB-0620', 'MB-0600', 'MB-0601', 'MB-0603', 'MB-0607', 'MB-0290', 'MB-0279', 'MB-0283', 'MB-0285', 'MB-0286', 'MB-0609', 'MB-0611', 'MB-0613', 'MB-0614', 'MB-0568', 'MB-0300', 'MB-0292', 'MB-0288', 'MB-0010', 'MB-0013', 'MB-0030', 'MB-0112', 'MB-0127', 'MB-0130', 'MB-0018', 'MB-0131', 'MB-0132', 'MB-0138', 'MB-0141', 'MB-0142', 'MB-0143', 'MB-0144', 'MB-0134', 'MB-0137', 'MB-0161', 'MB-0162', 'MB-0163', 'MB-0164', 'MB-0166', 'MB-0167', 'MB-0150', 'MB-0152', 'MB-0153', 'MB-0169', 'MB-0180', 'MB-0181', 'MB-0183', 'MB-0173', 'MB-0175', 'MB-0177', 'MB-0178', 'MB-0184', 'MB-0191', 'MB-0117', 'MB-0118', 'MB-0120', 'MB-0109', 'MB-0121', 'MB-0185', 'MB-0186', 'MB-0188', 'MB-0189', 'MB-0204', 'MB-0195', 'MB-0212', 'MB-0107', 'MB-0202', 'MB-0203', 'MB-0368', 'MB-0378', 'MB-0383', 'MB-0384', 'MB-0371', 'MB-0372', 'MB-0373', 'MB-0375', 'MB-0388', 'MB-0397', 'MB-0398', 'MB-0399', 'MB-0401', 'MB-0404', 'MB-0406', 'MB-0387', 'MB-0393', 'MB-0396', 'MB-0641', 'MB-0642', 'MB-0643', 'MB-0644', 'MB-0645', 'MB-0631', 'MB-0635', 'MB-0637', 'MB-0660', 'MB-0661', 'MB-0664', 'MB-0665', 'MB-0666', 'MB-0227', 'MB-0648', 'MB-0653', 'MB-0657', 'MB-0659', 'MB-0915', 'MB-0154', 'MB-0165', 'MB-0168', 'MB-0174', 'MB-0050', 'MB-0077', 'MB-0443', 'MB-0461', 'MB-0469', 'MB-0413', 'MB-0438', 'MB-0408', 'MB-0421', 'MB-0422', 'MB-0426', 'MB-0416', 'MB-0418', 'MB-0428', 'MB-0439', 'MB-0440', 'MB-0444', 'MB-0446', 'MB-0448', 'MB-0449', 'MB-0429', 'MB-0431', 'MB-0433', 'MB-0434', 'MB-0437', 'MB-0453', 'MB-0462', 'MB-0464', 'MB-0465', 'MB-0454', 'MB-0457', 'MB-0458', 'MB-0473', 'MB-0474', 'MB-0486', 'MB-0490', 'MB-0492', 'MB-0476', 'MB-0484', 'MB-0485', 'MB-0494', 'MB-0495', 'MB-0508', 'MB-0510', 'MB-0515', 'MB-0498', 'MB-0500', 'MB-0501', 'MB-0504', 'MB-0582', 'MB-0624', 'MB-0576', 'MB-0578', 'MB-0579', 'MB-0580', 'MB-0289', 'MB-0563', 'MB-0339', 'MB-0340', 'MB-0565', 'MB-0145', 'MB-0512', 'MB-0207', 'MB-0314', 'MB-0336', 'MB-0562', 'MB-0523', 'MB-0035', 'MB-0312', 'MB-0525', 'MB-0231', 'MB-0526', 'MB-0527', 'MB-0237', 'MB-0238', 'MB-0239', 'MB-0524', 'MB-0040', 'MB-0170', 'MB-0379', 'MB-0411', 'MB-0470', 'MB-0075', 'MB-0596', 'MB-0604', 'MB-0505', 'MB-0192', 'MB-0451', 'MB-0520', 'MB-0634', 'MB-0128', 'MB-0347', 'MB-0367', 'MB-0060', 'MB-0095', 'MB-0586', 'MB-0392', 'MB-0405', 'MB-2984', 'MB-2993', 'MB-2966', 'MB-2971', 'MB-2977', 'MB-2513', 'MB-2622', 'MB-2626', 'MB-2629', 'MB-2536', 'MB-2556', 'MB-2610', 'MB-2686', 'MB-2718', 'MB-3002', 'MB-3020', 'MB-2640', 'MB-2645', 'MB-2669', 'MB-2721', 'MB-2750', 'MB-2752', 'MB-2758', 'MB-2728', 'MB-2730', 'MB-2742', 'MB-2760', 'MB-2774', 'MB-2786', 'MB-3025', 'MB-2764', 'MB-2765', 'MB-2769', 'MB-2791', 'MB-2810', 'MB-2820', 'MB-3007', 'MB-3026', 'MB-2801', 'MB-2844', 'MB-2847', 'MB-2848', 'MB-2849', 'MB-2834', 'MB-2838', 'MB-2840', 'MB-2843', 'MB-2851', 'MB-2900', 'MB-2904', 'MB-2912', 'MB-2854', 'MB-2857', 'MB-2862', 'MB-2863', 'MB-2939', 'MB-2947', 'MB-2951', 'MB-2922', 'MB-2929', 'MB-2931', 'MB-2933', 'MB-3674', 'MB-3680', 'MB-3685', 'MB-3688', 'MB-3702', 'MB-3978', 'MB-3567', 'MB-3576', 'MB-3600', 'MB-3606', 'MB-3614', 'MB-3637', 'MB-3667', 'MB-3706', 'MB-3824', 'MB-3752', 'MB-3754', 'MB-3079', 'MB-3082', 'MB-3037', 'MB-3121', 'MB-3165', 'MB-3087', 'MB-3088', 'MB-3089', 'MB-3102', 'MB-3167', 'MB-3252', 'MB-3854', 'MB-3171', 'MB-3181', 'MB-3218', 'MB-3222', 'MB-3303', 'MB-3315', 'MB-3329', 'MB-3272', 'MB-3292', 'MB-3300', 'MB-3350', 'MB-3370', 'MB-3381', 'MB-3357', 'MB-3360', 'MB-3363', 'MB-3365', 'MB-3367', 'MB-3386', 'MB-3430', 'MB-3435', 'MB-3439', 'MB-3388', 'MB-3395', 'MB-3396', 'MB-3402', 'MB-3403', 'MB-3450', 'MB-3479', 'MB-3487', 'MB-3453', 'MB-3462', 'MB-3467', 'MB-3470', 'MB-3475', 'MB-3497', 'MB-3530', 'MB-3536', 'MB-3545', 'MB-3876', 'MB-3502', 'MB-3510', 'MB-3525', 'MB-3526', 'MB-3528', 'MB-0148', 'MB-0149', 'MB-0571', 'MB-0129'
],['MB-0247', 'MB-0342', 'MB-0343', 'MB-0249', 'MB-0880', 'MB-0354', 'MB-0349', 'MB-0002', 'MB-0005', 'MB-0291', 'MB-0309', 'MB-0241', 'MB-0242', 'MB-0255', 'MB-0333', 'MB-0536', 'MB-0066', 'MB-0101', 'MB-0006', 'MB-0085', 'MB-0216', 'MB-0590', 'MB-0622', 'MB-0602', 'MB-0608', 'MB-0914', 'MB-0612', 'MB-0616', 'MB-0287', 'MB-0126', 'MB-0139', 'MB-0122', 'MB-0206', 'MB-0208', 'MB-0200', 'MB-0386', 'MB-0395', 'MB-0629', 'MB-0647', 'MB-0194', 'MB-0389', 'MB-0538', 'MB-0394', 'MB-0419', 'MB-0425', 'MB-0415', 'MB-0441', 'MB-0467', 'MB-0471', 'MB-0459', 'MB-0460', 'MB-0488', 'MB-0475', 'MB-0479', 'MB-0519', 'MB-0574', 'MB-0583', 'MB-0625', 'MB-0575', 'MB-0910', 'MB-0119', 'MB-0577', 'MB-0654', 'MB-0890', 'MB-0381', 'MB-0921', 'MB-2617', 'MB-2564', 'MB-2613', 'MB-2614', 'MB-2708', 'MB-2641', 'MB-2771', 'MB-2772', 'MB-2793', 'MB-2796', 'MB-2828', 'MB-2858', 'MB-2927', 'MB-3804', 'MB-3823', 'MB-3707', 'MB-3711', 'MB-3747', 'MB-3062', 'MB-3035', 'MB-3355', 'MB-3361', 'MB-3427', 'MB-3436', 'MB-3389', 'MB-3488', 'MB-3495', 'MB-3452', 'MB-3459', 'MB-3466', 'MB-0328'
],['MB-0099', 'MB-0201', 'MB-0091', 'MB-0222', 'MB-0481', 'MB-0521', 'MB-0558', 'MB-0356', 'MB-0363', 'MB-0904', 'MB-0362', 'MB-0000', 'MB-0277', 'MB-0319', 'MB-0105', 'MB-0316', 'MB-0899', 'MB-0483', 'MB-0225', 'MB-0198', 'MB-0226', 'MB-0307', 'MB-0322', 'MB-0254', 'MB-0529', 'MB-0534', 'MB-0264', 'MB-0352', 'MB-0036', 'MB-0043', 'MB-0100', 'MB-0102', 'MB-0621', 'MB-0623', 'MB-0605', 'MB-0662', 'MB-0245', 'MB-0296', 'MB-0111', 'MB-0014', 'MB-0022', 'MB-0028', 'MB-0140', 'MB-0133', 'MB-0135', 'MB-0136', 'MB-0158', 'MB-0151', 'MB-0182', 'MB-0176', 'MB-0116', 'MB-0123', 'MB-0205', 'MB-0124', 'MB-0370', 'MB-0382', 'MB-0385', 'MB-0390', 'MB-0391', 'MB-0630', 'MB-0636', 'MB-0638', 'MB-0646', 'MB-0652', 'MB-0455', 'MB-0487', 'MB-0400', 'MB-0410', 'MB-0409', 'MB-0420', 'MB-0423', 'MB-0412', 'MB-0414', 'MB-0417', 'MB-0432', 'MB-0463', 'MB-0456', 'MB-0491', 'MB-0513', 'MB-0518', 'MB-0502', 'MB-0503', 'MB-0626', 'MB-0627', 'MB-0581', 'MB-0315', 'MB-0559', 'MB-0436', 'MB-0544', 'MB-0246', 'MB-0233', 'MB-0663', 'MB-0235', 'MB-0064', 'MB-0610', 'MB-0628', 'MB-0348', 'MB-0377', 'MB-2957', 'MB-2994', 'MB-2969', 'MB-2616', 'MB-2578', 'MB-2781', 'MB-3006', 'MB-2770', 'MB-2792', 'MB-2846', 'MB-2896', 'MB-3013', 'MB-2867', 'MB-2954', 'MB-2923', 'MB-3560', 'MB-3692', 'MB-3797', 'MB-3838', 'MB-3748', 'MB-3781', 'MB-3253', 'MB-3275', 'MB-3382', 'MB-3412', 'MB-3492', 'MB-3509']])

## Step 4: train classifier model

In [5]:
# initialize model
model_arch = "unet"
n_channels = dataset.n_channels
img_size = dataset.img_size
model = mp.PatchClassifier(n_channels, img_size, model_arch)

# train model
trainer_params = {
    "max_epochs": 30,
    "accelerator": "auto",
    "logger": False,
}
model = mp.train(
    model=model,
    dataset=dataset,
    predict_label=colname,
    trainer_params=trainer_params,
)

/central/home/zwang2/.cache/pypoetry/virtualenvs/morpheus-spatial-ndDQRg-x-py3.9/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /central/home/zwang2/.cache/pypoetry/virtualenvs/mor ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Training model with unet architecture


/central/home/zwang2/.cache/pypoetry/virtualenvs/morpheus-spatial-ndDQRg-x-py3.9/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /central/home/zwang2/.cache/pypoetry/virtualenvs/mor ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | predictor | Sequential | 11.5 M
-----------------------------------------
11.5 M    Trainable params
0         Non-trainable params
11.5 M    Total params
46.168    Total estimated model params size (MB)


Epoch 0: 100%|██████████| 556/556 [04:25<00:00,  2.09it/s, val_bce=0.569, val_precision=0.673, val_recall=0.384, val_bmc=0.439, val_auroc=0.673, val_f1=0.481, val_acc=0.868]

/central/home/zwang2/.cache/pypoetry/virtualenvs/morpheus-spatial-ndDQRg-x-py3.9/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


...

Epoch 29: 100%|██████████| 556/556 [00:27<00:00, 20.56it/s, val_bce=0.569, val_precision=0.618, val_recall=0.538, val_bmc=0.499, val_auroc=0.736, val_f1=0.570, val_acc=0.869]


Restoring states from the checkpoint path at /central/groups/mthomson/zwang2/IMC/output/danenbergBreast_sz48_pxl3_nc39/replicate/model/epoch=15-step=8896.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


model saved to /central/groups/mthomson/zwang2/IMC/output/danenbergBreast_sz48_pxl3_nc39/replicate/model/epoch=15-step=8896.ckpt


Loaded model weights from the checkpoint at /central/groups/mthomson/zwang2/IMC/output/danenbergBreast_sz48_pxl3_nc39/replicate/model/epoch=15-step=8896.ckpt


Testing DataLoader 0: 100%|██████████| 127/127 [00:50<00:00,  2.54it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8693965673446655
       test_auroc           0.7271588444709778
        test_bce            0.5681342482566833
        test_bmc            0.5189134478569031
         test_f1            0.5809020400047302
     test_precision         0.7011740803718567
       test_recall          0.5029512047767639
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [10]:
mp.test_model(
    dataset=dataset,
    predict_label=colname,
    model_path="/groups/mthomson/zwang2/IMC/output/danenbergBreast_sz48_pxl3_nc39/model/unet/lightning_logs/version_1/checkpoints/epoch=13-step=8120.ckpt",
    in_channels = dataset.n_channels,
    img_size = dataset.img_size,
)

Lightning automatically upgraded your loaded checkpoint from v1.8.6 to v2.2.2. To apply the upgrade to your files permanently, run `python -m lightning.pytorch.utilities.upgrade_checkpoint ../../../../../groups/mthomson/zwang2/IMC/output/danenbergBreast_sz48_pxl3_nc39/model/unet/lightning_logs/version_1/checkpoints/epoch=13-step=8120.ckpt`
/central/home/zwang2/.cache/pypoetry/virtualenvs/morpheus-spatial-ndDQRg-x-py3.9/lib/python3.9/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /central/home/zwang2/.cache/pypoetry/virtualenvs/mor ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing model at /groups/mthomson/zwang2/IMC/output/danenbergBreast_sz48_pxl3_nc39/model/unet/lightning_logs/version_1/checkpoints/epoch=13-step=8120.ckpt
Testing DataLoader 0: 100%|██████████| 127/127 [00:03<00:00, 40.11it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8533251285552979
       test_auroc           0.7240810394287109
        test_bce             0.577873170375824
        test_bmc            0.47906404733657837
         test_f1            0.5576087832450867
     test_precision         0.6215755939483643
       test_recall          0.5191249847412109
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


## Step 5: generate counterfactuals using trained classifier

In [ ]:
# images to generate counterfactuals
dataset.get_split_info()
select_metadata = dataset.metadata[
    (dataset.metadata["Contains_Tumor"] == 1)
    & (dataset.metadata["Contains_Tcytotoxic"] == 0)
    & (dataset.metadata["splits"] == "train")
]

In [ ]:
# channels allowed to be perturbed
channel_to_perturb = [
    "Glnsynthetase",
    "CCR4",
    "PDL1",
    "LAG3",
    "CD105endoglin",
    "TIM3",
    "CXCR4",
    "PD1",
    "CYR61",
    "CD44",
    "IL10",
    "CXCL12",
    "CXCR3",
    "Galectin9",
    "YAP",
]

# probability cutoff for classification
threshold = 0.43

# optimization parameters
optimization_param = {
    "use_kdtree": True,
    "theta": 40.0,
    "kappa": (threshold - 0.5) * 2,
    "learning_rate_init": 0.1,
    "beta": 80.0,
    "max_iterations": 1000,
    "c_init": 1000.0,
    "c_steps": 5,
    "numerical_diff": False,
}

# example of selected instances to generate counterfactuals
print(f"Number of selected instances: {len(select_metadata)}")
print(select_metadata.head())

In [ ]:
# Generate counterfactuals using trained model
mp.get_counterfactual(
    images=select_metadata.iloc[:1],
    dataset=dataset,
    target_class=1,
    model_path=f"{dataset.root_dir}/model/checkpoints/epoch=42-step=13287.ckpt",
    channel_to_perturb=channel_to_perturb,
    optimization_params=optimization_param,
    threshold=threshold,
    save_dir=f"{dataset.root_dir}/cf/",
    device="cpu",
    num_workers=1,
    verbosity=0,
    model_kwargs={"in_channels": n_channels, "img_size": img_size},
)